In [ ]:
# ==============================================================================
# NOTEBOOK 1: Create and Save Merged Model
# ==============================================================================

# --- CELL 1: Setup ---
import os
# Force the environment to see only one GPU to guarantee stability
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Install all necessary libraries
!pip install "transformers==4.54.1" -qU
!pip install "wandb>=0.17.0" -qU
!pip install --no-deps "bitsandbytes>=0.43.1" "accelerate>=0.31.0" "xformers==0.0.29.post3" "trl>=0.9.4" triton -q
!pip install --force-reinstall --no-deps git+https://github.com/unslothai/unsloth.git -q
!pip install --force-reinstall --no-deps git+https://github.com/unslothai/unsloth-zoo.git -q
!pip install -U peft -q
!pip install "timm>=1.0.16" -qU

# --- CELL 2: W&B Login ---
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb_api_key")
wandb.login(key=wandb_api_key)

# --- CELL 3: Download Adapters ---
run = wandb.init(project="mc-s-&-e-n")
artifact_path = 'jdmasciano2-university-of-lagos/mc-s-&-e-n/maize-adapters-icy-sweep-2:v0'
artifact = run.use_artifact(artifact_path, type='model')
champion_adapters_path = artifact.download()
run.finish()


# ==============================================================================
# CELL 4: Load, Configure PEFT, and Save Merged with the Official Unsloth Method
# ==============================================================================
from unsloth import FastVisionModel
import torch

# The full, 4B parameter model ID from Unsloth
MODEL_NAME_E4B = "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit"
# Define a clean output path for Colab
merged_model_path = "/kaggle/working/AuraMind-E4B-Finetuned-Merged/"

print("--- Step 2: Loading Full E4B Base Model in 4-bit ---")
model, tokenizer = FastVisionModel.from_pretrained(
    model_name = MODEL_NAME_E4B,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    device_map = {"": "cuda:0"}
)
print("✅ Base model loaded.")

# --- Step 3: MUST Apply PEFT Configuration to the Model FIRST ---
# This is critical. It wraps the model in the PeftModel class.
print("\n--- Step 3: Applying PEFT Configuration to the Model ---")
model = FastVisionModel.get_peft_model(
    model,
    r = 16, # Must match your champion adapter's config
    lora_alpha = 16, # Must match your champion adapter's config
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    finetune_vision_layers = True, # Must match your champion adapter's config
    finetune_language_layers = True,
)
print("✅ PEFT configuration applied.")

print("\n--- Step 4: Loading Our Champion Adapter Weights ---")
# Now, we load our trained weights into the correctly configured PEFT model.
model.load_adapter(champion_adapters_path, adapter_name="default")
print("✅ Champion LoRA adapters loaded.")

# --- Step 5: Merge and Save Using the OFFICIAL Unsloth Function ---
# This single function handles the merge and save operation correctly for 4-bit models.
# It replaces both `merge_and_unload` and `save_pretrained`.
print(f"\n--- Step 5: Merging and Saving to Float16 ---")
model.save_pretrained_merged(merged_model_path, tokenizer, save_method="float16")
print(f"✅ Merged model successfully saved to: {merged_model_path}")

In [ ]:
# ==============================================================================
# CELL 5: Strategic Cleanup
# ==============================================================================
import shutil
import os

print("--- Starting final cleanup ---")

# List everything in the /kaggle/working/ directory
for item in os.listdir('/kaggle/working/'):
  item_path = os.path.join('/kaggle/working/', item)
  # Check if it's NOT our target directory
  if item != 'AuraMind-E4B-Finetuned-Merged':
    print(f"Deleting: {item_path}")
    # Use shutil.rmtree for directories and os.remove for files
    if os.path.isdir(item_path):
      shutil.rmtree(item_path)
    else:
      os.remove(item_path)

print("\n✅ Cleanup complete. Only the merged model folder remains.")
!ls -l /kaggle/working/